In [ ]:
import pandas as pd
import re
from glob import glob
from tqdm import tqdm

In [ ]:
df = []

In [ ]:
import hashlib
import os

# menyiapkan algoritma hashing untuk menyamarkan ip pengguna
def hash_ip(ip_address):
    # Concatenate IP address and salt
    ip = ip_address.encode()
    # Create a SHA-256 hash of the IP address with salt
    hashed_ip = str(hashlib.sha3_224(ip).hexdigest())
    return hashed_ip

In [ ]:
# data webapi_extracted sementara hanya untuk di run di server lokal sehingga
# datanya tidak tersedia di colab dan hanya bisa dijalankan di server lokal
# karena memuat data IP public pengguna

# kode ini melakukan preprocessing sekaligus bind IP address
# output dari ./webapi_extracted/* = tes.csv

In [ ]:
for path in tqdm(glob("./webapi_extracted/*")):
    file1 = open(path)
    teks = file1.readlines()
    teks = [x for x in teks if "v1/api" in x]
    for log in teks:
        try:
            ip_pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
            time_pattern = r'\[(.*?)\]'
            get_pattern = r'\] "(.*?) HTTP\/1\.1\"'

            # Extract IP address
            ip_address = re.search(ip_pattern, log).group(1)

            # Extract time
            time = re.search(time_pattern, log).group(1)

            # Extract GET request
            get_request = re.search(get_pattern, log).group(1)

            # cek = (ip_address.count(".") == 3)

            df.append({
                "ip": hash_ip(ip_address),
                "time": time,
                "request": get_request
            })
        except Exception as e:
            pass

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:04<00:00,  7.19it/s]


In [ ]:
df[0:5]

[{'ip': '98b1bd0d251b909d94a8a359f2c2a37c5f7bc5fc3ab91bf768050d0c',
  'time': '30/Apr/2024:23:13:37 +0700',
  'request': 'GET /v1/api/list/model/data?domain=0000&var=1470&th=123&vervar=249&key=06186138228a67b7a37bfb2730be3c2b'},
 {'ip': '98b1bd0d251b909d94a8a359f2c2a37c5f7bc5fc3ab91bf768050d0c',
  'time': '30/Apr/2024:23:13:38 +0700',
  'request': 'GET /v1/api/list?model=data&domain=0000&var=1189&vervar=9999&key=ae6caa2538ecda3d5fe7be228f17ee7d'},
 {'ip': 'ab0000895a045dfba2b9340365a32cd88120309ab5457e4f7c55766d',
  'time': '30/Apr/2024:23:18:05 +0700',
  'request': 'GET /v1/api/list/model/subject/lang/ind/domain/1900/subcat/1/key/b2da99d6cd045241b5a07b3b3009549c/'},
 {'ip': 'ab0000895a045dfba2b9340365a32cd88120309ab5457e4f7c55766d',
  'time': '30/Apr/2024:23:18:05 +0700',
  'request': 'GET /v1/api/list/model/subject/lang/ind/domain/1900/subcat/2/key/b2da99d6cd045241b5a07b3b3009549c/'},
 {'ip': 'ab0000895a045dfba2b9340365a32cd88120309ab5457e4f7c55766d',
  'time': '30/Apr/2024:23:18:06 

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df

,ip,time,request
0,98b1bd0d251b909d94a8a359f2c2a37c5f7bc5fc3ab91b...,30/Apr/2024:23:13:37 +0700,GET /v1/api/list/model/data?domain=0000&var=14...
1,98b1bd0d251b909d94a8a359f2c2a37c5f7bc5fc3ab91b...,30/Apr/2024:23:13:38 +0700,GET /v1/api/list?model=data&domain=0000&var=11...
2,ab0000895a045dfba2b9340365a32cd88120309ab5457e...,30/Apr/2024:23:18:05 +0700,GET /v1/api/list/model/subject/lang/ind/domain...
3,ab0000895a045dfba2b9340365a32cd88120309ab5457e...,30/Apr/2024:23:18:05 +0700,GET /v1/api/list/model/subject/lang/ind/domain...
4,ab0000895a045dfba2b9340365a32cd88120309ab5457e...,30/Apr/2024:23:18:06 +0700,GET /v1/api/list/model/subject/lang/ind/domain...
...,...,...,...
401746,aeeb956f2e4ca40078e5671effa603e87148af0380c3f2...,31/May/2024:23:13:24 +0700,GET /v1/api/list?model=var&lang=ind&domain=940...
401747,aeeb956f2e4ca40078e5671effa603e87148af0380c3f2...,31/May/2024:23:13:32 +0700,GET /v1/api/list?model=var&lang=ind&domain=940...
401748,aeeb956f2e4ca40078e5671effa603e87148af0380c3f2...,31/May/2024:23:13:51 +0700,GET /v1/api/list?model=var&lang=ind&domain=940...
401749,aeeb956f2e4ca40078e5671effa603e87148af0380c3f2...,31/May/2024:23:14:03 +0700,GET /v1/api/list?model=var&lang=ind&domain=940...


In [ ]:
# ip di factorize agar tidak kepanjangan
df['ip'] = pd.factorize(df['ip'])[0]
df['ip'] = [f'ip-{x}' for x in df.ip]

In [ ]:
def format_datetime(date_str):
    try:
        date_format = "%d/%b/%Y:%H:%M:%S %z"
        return(pd.to_datetime(date_str, format=date_format))
    except Exception:
        return(None)

In [ ]:
df['time'] = df['time'].apply(format_datetime)

In [ ]:
df

,ip,time,request
0,ip-0,2024-04-30 23:13:37+07:00,GET /v1/api/list/model/data?domain=0000&var=14...
1,ip-0,2024-04-30 23:13:38+07:00,GET /v1/api/list?model=data&domain=0000&var=11...
2,ip-1,2024-04-30 23:18:05+07:00,GET /v1/api/list/model/subject/lang/ind/domain...
3,ip-1,2024-04-30 23:18:05+07:00,GET /v1/api/list/model/subject/lang/ind/domain...
4,ip-1,2024-04-30 23:18:06+07:00,GET /v1/api/list/model/subject/lang/ind/domain...
...,...,...,...
401746,ip-2,2024-05-31 23:13:24+07:00,GET /v1/api/list?model=var&lang=ind&domain=940...
401747,ip-2,2024-05-31 23:13:32+07:00,GET /v1/api/list?model=var&lang=ind&domain=940...
401748,ip-2,2024-05-31 23:13:51+07:00,GET /v1/api/list?model=var&lang=ind&domain=940...
401749,ip-2,2024-05-31 23:14:03+07:00,GET /v1/api/list?model=var&lang=ind&domain=940...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401751 entries, 0 to 401750
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype                    
---  ------   --------------   -----                    
 0   ip       401751 non-null  object                   
 1   time     400771 non-null  datetime64[ns, UTC+07:00]
 2   request  401751 non-null  object                   
dtypes: datetime64[ns, UTC+07:00](1), object(2)
memory usage: 9.2+ MB


In [ ]:
df['request'][0]

'GET /v1/api/list/model/data?domain=0000&var=1470&th=123&vervar=249&key=06186138228a67b7a37bfb2730be3c2b'

In [ ]:
df['request'][100]

'GET /v1/api/list/model/data/domain/0000/var/1901/key/5fd13ade2b185833c122a9939cd09f59'

In [ ]:
def format_requests(str):
    try:
        # str = 'GET /v1/api/list/model/data/domain/0000/var/1901/key/5fd13ade2b185833c122a9939cd09f59'
        str = re.sub(r'key.*', '', str)
        str = re.sub(r'GET /v1/api/list', '', str)
        if "GET" in str or "- -" in str or "model" not in str:
            return(None)
        else:
            str = str.replace("?", "/")
            str = str.replace("&", "/")
            str = str.replace("=", "/")
            str = str.replace("//", "")
            if str == "/" or " " in str:
                return(None)
            else:
                return(str)
    except Exception:
        return(None)

In [ ]:
df['request']=df['request'].apply(format_requests)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.to_csv("./tes.csv", index=False)

In [ ]:
# df[0:200].to_csv("./tes0.csv", index=False)